In [31]:
import numpy as np
import os
import pandas as pd
import pykonal

In [55]:
class Namespace(object):
    def __init__(self):
        pass
    
    
def parse_clargs():
    """
    Parse and return command-line arguments.
    """
    clargs = Namespace()
    clargs.detections_file = "data/small_test/detections.h5"
    clargs.stations_file = "data/small_test/stations.h5"
    clargs.traveltime_directory = "data/small_test/traveltimes"
    clargs.pwave_velocity_file = "data/small_test/coso_1d_vp.npz"
    clargs.swave_velocity_file = "data/small_test/coso_1d_vs.npz"
    clargs.output_file = "output/small_test/results.h5"
    return (clargs)

def parse_params():
    """
    Parse and return algorithm parameters.
    """
    params = dict()
    params["k_max"] = 10
    return (params)

    
def main():
    # Parse command-line arguments.
    clargs = parse_clargs()
    
    # Parse algorithm parameters.
    params = parse_params()
    
    # Load and preprocess input data.
    detections = load_detections(clargs.detections_file)
    stations = load_stations(clargs.stations_file)
    pwave_vmodel = load_velocity_model(clargs.pwave_velocity_file)
    swave_vmodel = load_velocity_model(clargs.swave_velocity_file)
    
    # Compute traveltime lookup tables.
    compute_traveltime_lookup_tables(
        clargs.traveltime_directory,
        detections, 
        stations, 
        pwave_vmodel, 
        swave_vmodel
    )
    
    # Iterate over trial values of k.
    for k in range(1, params["k_max"]):
    
        # Generate initial means.
        means = initialize_means(k, detections, stations)
        # Initialize convergence flag.
        has_converged = False
        
        # Iteratively update means and associations until convergence.
        while not has_converged:
            # Update associations
            detections = update_associations(detections, means, traveltime_dir)
            # Update means
            means = update_means(detections, traveltime_dir)
        
        # Compute the Akaike Information Criterion
        aic = aic(detections, means, travltime_dir)
        # Store results for post-processing.
        write_results(means, aic, clargs.output_file)
        
    return (True)


def aic(detections, means, travltime_dir):
    """
    """
    raise (NotImplementedError("aic()"))
    

def compute_traveltime_lookup_tables(
    traveltime_dir, 
    detections, 
    stations, 
    pwave_vmodel,
    swave_vmodel, 
    overwrite=False,
    verbose=True
):
    """
    Compute traveltime lookup tables (optionally overwriting existing
    tables) for all detections.
    """
    # Make the output directory if it doesn't exist.
    os.makedirs(traveltime_dir, exist_ok=True)
    # Drop duplicated (network, station, phase) triplets.
    detections = detections.drop_duplicates(["network", "station", "phase"])
    detections = detections[["network", "station", "phase"]].values
    # Set the index of stations DataFrame to ("network", "station")
    stations = stations.set_index(["network", "station"])
    for network, station, phase in detections:
        path = os.path.join(traveltime_dir, f"{network}.{station}.{phase}.npz")
        if overwrite or not os.path.exists(path):
            if verbose:
                print(
                    f"Computing {phase}-wave traveltime lookup table for "
                    f"{network}.{station}."
                )
            vmodel = pwave_vmodel if phase == "P" else swave_vmodel
            keys = ["latitude", "longitude", "depth"]
            lat, lon, depth = stations.loc[(network, station), keys]
            solver = pykonal.solver.PointSourceSolver(coord_sys="spherical")
            solver.vv.min_coords = vmodel.min_coords
            solver.vv.node_intervals = vmodel.node_intervals
            solver.vv.npts = vmodel.npts
            solver.vv.values = vmodel.values
            solver.src_loc = pykonal.transformations.geo2sph([lat, lon, depth])
            solver.solve()
            solver.tt.savez(path)


def initialize_means(k_max, detections, stations):
    """
    Initialize means for clustering.
    """
    raise (NotImplementedError("initialize_means()"))
    
    
def load_detections(detections_file):
    """
    Read, preprocess, and return detections from file.
    
    Input file should be a pandas.HDFStore file with a 'detections'
    table comprising 'network', 'station', 'phase', and 'time' fields.
    """
    with pd.HDFStore(detections_file, mode="r") as store:
        detections = store["detections"]
    return (detections)


def load_stations(stations_file):
    """
    Read, preprocess, and return stations from file.
    
    Input file should be a pandas.HDFStore file with a 'stations' table
    comprising 'network', 'station', 'latitude', 'longitude', and
    'elevation' fields.
    """
    with pd.HDFStore(stations_file, mode="r") as store:
        stations = store["stations"]
    stations["depth"] = -stations["elevation"]
    stations = stations.drop(columns=["elevation"])
    return (stations)


def load_velocity_model(velocity_file):
    """
    Read velocity file and return as pykonal.fields.ScalarField3D object.
    """
    vmodel = pykonal.fields.load(velocity_file)
    return (vmodel)


def update_associations(detections, means, traveltime_dir):
    """
    """
    raise (NotImplementedError("update_associations()"))

    
def update_means(detections, traveltime_dir):
    """
    """
    raise (NotImplementedError("update_means()"))
    

def write_results(means, aic, output_file):
    """
    """
    raise (NotImplementedError("write_results()"))

In [56]:
main()

NotImplementedError: initialize_means()